<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/llm/opencalm3b_finetuning_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lora Instruct

Installation

In [ ]:
%cd /content
!git clone https://github.com/leehanchung/lora-instruct
%cd lora-instruct
!pip install poetry
!poetry export --without-hashes --with dev --output requirements.txt
!pip install -r requirements.txt

Finetuning

In [ ]:
%cd /content/lora-instruct
!python finetune.py \
    --base_model 'cyberagent/open-calm-3b' \
    --data_path 'kunishou/databricks-dolly-15k-ja' \
    --output_dir './lora-alpaca' \
    --batch_size 16 \
    --micro_batch_size 2 \
    --num_epochs 3 \
    --learning_rate 3e-4 \
    --cutoff_len 512 \
    --val_set_size 2000 \
    --lora_r 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --train_on_inputs \
    --add_eos_tokens \
    --group_by_length


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [11]:
!cp -r /content/lora-instruct/lora-alpaca /content/drive/MyDrive

inference

https://note.com/npaka/n/n227771a2527c

In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# トークナイザーとモデルの準備
tokenizer = AutoTokenizer.from_pretrained(
    "cyberagent/open-calm-3b"
)
model = AutoModelForCausalLM.from_pretrained(
    "cyberagent/open-calm-3b", 
    torch_dtype=torch.float16
)

In [ ]:
from peft import PeftModel

# PEFTモデルの追加
model = PeftModel.from_pretrained(model, "/content/lora-instruct/lora-alpaca", device_map="auto")
model = model.to("cuda:0")

In [14]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:
"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:
"""

In [ ]:
prompt = generate_prompt("明日の天気を教えてほしい。")
inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
input_length = inputs.input_ids.shape[1]

with torch.no_grad():
  outputs = model.generate(
      **inputs, 
      max_new_tokens=128, 
      do_sample=True, 
      temperature=0.7, 
      top_p=0.7, 
      top_k=50, 
      return_dict_in_generate=True
  )
token = outputs.sequences[0, input_length:]
output_str = tokenizer.decode(token)

# 確認
print(output_str)

alpaca lora finetuning

In [ ]:
!git clone https://github.com/tloen/alpaca-lora.git
%cd alpaca-lora
!pip install -r requirements.txt

In [ ]:
"""
batch_size=128,
micro_batch_size=2,  # gradient_accumulation_steps=64
num_epochs=3,
learning_rate=3e-4,
lora_r=8,
lora_alpha=16,
lora_dropout=5e-2,
lora_target_modules=['query_key_value'],
train_on_inputs=True,
add_eos_tokens=True,
warmup_steps=100,
"""

In [ ]:
!python finetune.py \
    --base_model 'cyberagent/open-calm-1b' \
    --data_path 'kunishou/databricks-dolly-15k-ja' \
    --output_dir './lora-alpaca' \
    --batch_size 4 \
    --micro_batch_size 1 \
    --num_epochs 3 \
    --learning_rate 3e-4 \
    --cutoff_len 512 \
    --val_set_size 2000 \
    --lora_r 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target_modules 'query_key_value' \
    --train_on_inputs \
    --add_eos_tokens \
    --group_by_length